## Detector Training
This notebook consist of training YOLOv8 with ultralystic API (PyTorch) to detect meter exclusively in image

In [ ]:
import os
import cv2
import pickle
import matplotlib.pyplot as plt

In [ ]:
# downloading and extraction paddle dataset
det = 'https://bj.bcebos.com/paddlex/examples/meter_reader/datasets/meter_det.tar.gz'

if ('meter_seg' not in os.listdir()):
    for i in [det]:
        !wget $i
        dir_name = i.split('/')[-1]
        !tar -xf $dir_name
else:
    print('Directory already exist')

### convering annotations from JSON to YOLO supportable format

In [ ]:
!cp -a meter_det/train datasets/data/train/

In [ ]:
!cp -a meter_det/test datasets/data/validation

In [ ]:
import json

In [ ]:
with open('/content/meter_det/annotations/instance_test.json') as file:
    annotations = json.load(file)

In [ ]:
images = annotations['images']
annotation = annotations['annotations']

In [ ]:
img_datas = { i['id'] : [i['file_name'][:-4],i['width'],i['height']] for i in images}

In [ ]:
bboxs = {}
for i in annotation:
    img_id = i['image_id']
    if img_id not in bboxs:
        bboxs[img_id] = [i['bbox']]
    else:
        bboxs[img_id].append(i['bbox'])

In [ ]:
final_data = {}

for i in list(img_datas.keys()):
    img_data = img_datas[i]
    bbox = bboxs[i]
    bbox_processed = []
    for j in bbox:
        j[0] = j[0] + (j[2]/2)
        j[1] = j[1] + (j[3]/2)
        b = []
        for n,k in enumerate(j):
            if n%2==0:
                b.append(k/img_data[1])
            else:
                b.append(k/img_data[2])
        bbox_processed.append(b)
    
    final_data[img_data[0]] = [bbox_processed,img_data[1],img_data[2]]

In [ ]:
for i in list(final_data.keys()):
    data = final_data[i]
    write = ''
    for j in data[0]:
        write += '0 '
        for k in j:
            write += str(k)
            write += ' '
        write += '\n'
    with open(f'datasets/data/validation/labels/{i}.txt','w') as f:
        f.write(write)

### Starting custom training

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO

In [ ]:
model = YOLO('yolov8n.yaml') #training from scratch
model.train(data='data.yaml',epochs=10,imgsz=512)


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.Conv                  [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.Conv                  [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.C2f                   [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.Conv                  [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.C2f                   [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.Conv                  [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.C2f                   [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.Conv                  [128

  0%|          | 0.00/755k [00:00<?, ?B/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.Conv                  [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.Conv                  [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.C2f                   [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.Conv                  [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.C2f                   [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.Conv                  [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.C2f                   [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

  0%|          | 0.00/6.23M [00:00<?, ?B/s]

AMP: checks passed ✅
optimizer: SGD(lr=0.01) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias
train: Scanning /content/datasets/data/train/labels... 725 images, 0 backgrounds, 0 corrupt: 100%|██████████| 725/725 [00:00<00:00, 1047.47it/s]
train: New cache created: /content/datasets/data/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))
val: Scanning /content/datasets/data/validation/labels... 58 images, 0 backgrounds, 0 corrupt: 100%|██████████| 58/58 [00:00<00:00, 658.89it/s]
val: New cache created: /content/datasets/data/validation/labels.cache
Plotting labels to runs/detect/train/labels.jpg... 
Image sizes 512 train, 512 val
Using 2 dataloader workers
Logging results to runs/detect/train
Starting training for 10 epochs...
Closing dataloader mosaic
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(

In [ ]:
from google.colab import files
files.download("/content/runs/detect/train/weights/best.pt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>